# Predicting Flight Delay

Problem Set-up:
We define a delayed flight to be one that is delayed by >= 15 minutes. 
The prediction problem is to train a model that can classify flights, to predict if they will or will not be delayed.

Use case:
- The idea is that this model would be useful to choosing airlines, flightpaths, airports, at the time of booking, relatively in advance of the scheduled departure (days, weeks, months ahead of time). Therefore, the prediction problem will focus on features that can be known in advance, rather than predicting using day-off features like weather and previous flights from that day. 

Notes:
- We restrict the analysis to relatively large airport, those with more than 20 (domestic) flights a day

# Create separate models to predict for each airport 

Motivation:
The weights that should eb put on features (e.g. airlines) may differ depending on the airport (see e.g. NB 4B) since different airports can have different environments (e.g. San Diego and Chicago winters are very different; American Airlines is better in Tuscon than in Anchorage.

In [1]:
# Imports
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
sns.set_style('white')

from sklearn.linear_model import LogisticRegression
from sklearn. metrics import roc_auc_score

# Load data

In [2]:
# Import custom code
from flightdelay.fld import io as flio
airlines_df, airports_df, flights_df = flio.load_data()

In [3]:
# Filter data to keys of interest
keys = ['MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT',
       'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME',
       'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME',
       'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN',
       'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY']
flights_df = flights_df[keys]

# Remove airports with less than a certain number of flights

In [4]:
all_airports, airport_inverse, airport_count = np.unique(flights_df['ORIGIN_AIRPORT'],return_counts=True,return_inverse=True)

In [5]:
# Determine number of flights for the origin airport
Nflights_orig = np.zeros(len(airport_inverse))
for i in range(len(all_airports)):
    Nflights_orig[np.where(airport_inverse==i)] = airport_count[i]

In [6]:
flights_df = flights_df.loc[flights_df.index[Nflights_orig>=7300]]

# Remove cancelled flights

In [7]:
flights_df = flights_df.dropna()
flights_df.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY
0,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,...,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0
1,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,...,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0
2,1,1,4,US,840,N171US,SFO,CLT,20,18.0,...,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0
3,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,...,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0
4,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,...,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0


# Compute features
* one-hot day of week
* one-hot month
* one-hot airline
* one-hot airport
* one-hot departure hour

### 0. Original feat

In [8]:
def make_onehot_feat_dict_from_vals(df, feat_key, feat_name, feat_cutoffs):
    
    # Create keys
    N_feat = len(feat_cutoffs) - 1
    keys = [0]*N_feat
    for i in range(N_feat):
        keys[i] = 'f_'+feat_name+'_'+ str(feat_cutoffs[i])
    # Find the indices for each class
    feat_dict = {}
    for i in range(N_feat):
        feat_dict[keys[i]] = np.transpose(
                    np.logical_and(df[feat_key].values>= feat_cutoffs[i],
                                df[feat_key].values<feat_cutoffs[i+1]))
    return feat_dict

In [9]:
cutoffs = np.arange(60,780,60)
original_feat = make_onehot_feat_dict_from_vals(flights_df,
                    'SCHEDULED_TIME', 'dur', cutoffs)

### 1. Day of week

In [10]:
def make_onehot_feat_dict(df, feat_key, feat_name):
    # Create features for each day of the week
    feat_vals = df[feat_key].values
    all_vals = np.unique(feat_vals)
    N_vals = len(all_vals)
    N_feat = N_vals - 1

    # Create keys
    keys = [0]*N_feat
    for i in range(N_feat):
        keys[i] = 'f_'+feat_name+'_'+ str(all_vals[i])

    # Create value for each training example in dict
    feat_dict = {}
    for i, k in enumerate(keys):
        this_day = all_vals[i]
        feat_dict[k] = feat_vals == this_day
    return feat_dict

In [11]:
daysfeat_dict = make_onehot_feat_dict(flights_df, 'DAY_OF_WEEK', 'day')

### 2. Month

In [12]:
monthsfeat_dict = make_onehot_feat_dict(flights_df, 'MONTH', 'month') 

### 3. Departing airport

In [13]:
dapfeat_dict = make_onehot_feat_dict(flights_df, 'ORIGIN_AIRPORT', 'dap') 

### 4. Airline

In [15]:
alfeat_dict = make_onehot_feat_dict(flights_df, 'AIRLINE', 'al') 

### 5. Departure hour

In [16]:
# Add departure hour as a feature
flights_df['HOUR_DEPARTURE'] = np.floor(flights_df['SCHEDULED_DEPARTURE'].values/100).astype(int)
hrfeat_dict = make_onehot_feat_dict(flights_df, 'HOUR_DEPARTURE', 'hr')

# Save new feature matrix

In [17]:
all_dicts = [original_feat, daysfeat_dict, monthsfeat_dict, dapfeat_dict, alfeat_dict, hrfeat_dict]
feat_dict = all_dicts[0].copy()
for d in all_dicts[1:]:
    feat_dict.update(d)

In [18]:
df_feat = pd.DataFrame.from_dict(feat_dict)
df_feat.head()

,f_al_AA,f_al_AS,f_al_B6,f_al_DL,f_al_EV,f_al_F9,f_al_HA,f_al_MQ,f_al_NK,f_al_OO,...,f_month_10,f_month_11,f_month_2,f_month_3,f_month_4,f_month_5,f_month_6,f_month_7,f_month_8,f_month_9
0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Make train val and test sets

In [23]:
N_flights = 1000000#len(flights_df)
N_train = int(N_flights*.5)
N_val = int(N_flights*.25)
N_test = N_flights - N_train - N_val

In [28]:
# Shuffle data
np.random.seed(0)
flight_shuff_idx = np.random.permutation(df_feat.index)
df_shuffle = df_feat.loc[flight_shuff_idx]

labels_preshuffle = flights_df['DEPARTURE_DELAY'].values
labels_shuffle = labels_preshuffle[flight_shuff_idx]
df_shuffle = df_shuffle.reset_index(drop=True)
df_shuffle.head()

,f_al_AA,f_al_AS,f_al_B6,f_al_DL,f_al_EV,f_al_F9,f_al_HA,f_al_MQ,f_al_NK,f_al_OO,...,f_month_10,f_month_11,f_month_2,f_month_3,f_month_4,f_month_5,f_month_6,f_month_7,f_month_8,f_month_9
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
5,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
7,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [30]:
delay_cutoff = 15

y_train = labels_shuffle[:N_train] > delay_cutoff
y_val = labels_shuffle[N_train:N_train+N_val] > delay_cutoff
y_test = labels_shuffle[N_train+N_val:N_train+2*N_val] > delay_cutoff

In [31]:
# zscore all features
from scipy.stats import zscore
df_shuffle = df_shuffle.apply(zscore)

In [32]:
X_train = df_shuffle.loc[np.arange(N_train)]
X_val = df_shuffle.loc[np.arange(N_train,N_train+N_val)]
X_test = df_shuffle.loc[np.arange(N_train+N_val,N_train+2*N_val)]

# 1. Train whole model

In [33]:
# Run model for all individual feature sets
all_C = [.00001, .001, 1000, 100000]
models = np.zeros(len(all_C),dtype=object)
train_aucs = np.zeros(len(all_C))
val_aucs = np.zeros(len(all_C))
test_aucs = np.zeros(len(all_C))
for i, c in enumerate(all_C):
    print(i)
    # Train model
    models[i] = LogisticRegression(C=c)
    models[i].fit(X_train, y_train)

    # Evaluate model
    train_aucs[i] = roc_auc_score(y_train, models[i].predict_proba(X_train)[:,1])
    val_aucs[i] = roc_auc_score(y_val, models[i].predict_proba(X_val)[:,1])
    test_aucs[i] = roc_auc_score(y_test, models[i].predict_proba(X_test)[:,1])

0
1
2
3
4


In [35]:
print(train_aucs)
print(val_aucs)
print(test_aucs)

[ 0.67311686  0.67312198  0.67312158  0.67312151  0.6731215 ]
[ 0.66779051  0.66779498  0.66779525  0.66779529  0.66779529]
[ 0.67299547  0.6729959   0.6729947   0.67299448  0.67299449]


# 2. Train models with individual onehot feature sets

In [36]:
feat_sets = [0]*len(all_dicts)
for i in range(len(all_dicts)):
    feat_sets[i] = list(all_dicts[i].keys())

In [38]:
# Run model for all individual feature sets
all_C = [.01, .1, 1, 10, 100]
models = np.zeros((len(feat_sets),len(all_C)),dtype=object)
train_aucs = np.zeros((len(feat_sets),len(all_C)))
val_aucs = np.zeros((len(feat_sets),len(all_C)))
test_aucs = np.zeros((len(feat_sets),len(all_C)))
for i, ks in enumerate(feat_sets):
    for j, c in enumerate(all_C):
        print(i,j)
        # Train model
        models[i,j] = LogisticRegression(C=c)
        models[i,j].fit(X_train[ks], y_train)
        
        # Evaluate model
        train_aucs[i,j] = roc_auc_score(y_train, models[i,j].predict_proba(X_train[ks])[:,1])
        val_aucs[i,j] = roc_auc_score(y_val, models[i,j].predict_proba(X_val[ks])[:,1])
        test_aucs[i,j] = roc_auc_score(y_test, models[i,j].predict_proba(X_test[ks])[:,1])

0 0
0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4
2 0
2 1
2 2
2 3
2 4
3 0
3 1
3 2
3 3
3 4
4 0
4 1
4 2
4 3
4 4
5 0
5 1
5 2
5 3
5 4


In [43]:
print(feat_sets[0][0])
print(feat_sets[1][0])
print(feat_sets[2][0])
print(feat_sets[3][0])
print(feat_sets[4][0])
print(feat_sets[5][0])

f_dur_60
f_day_1
f_month_1
f_dap_ABQ
f_al_AA
f_hr_0


In [40]:
print(train_aucs)
print(val_aucs)
print(test_aucs)

[[ 0.51725754  0.51725754  0.51725754  0.51725754  0.51725754]
 [ 0.51754386  0.51754386  0.51754386  0.51754386  0.51754386]
 [ 0.56460727  0.56460727  0.56460727  0.56460727  0.56460727]
 [ 0.56108272  0.56108272  0.56108272  0.56108272  0.56108272]
 [ 0.56894006  0.56894006  0.56894006  0.56894006  0.56894006]
 [ 0.62984175  0.62985202  0.62985202  0.62985202  0.62985202]]
[[ 0.51628773  0.51628773  0.51628773  0.51628773  0.51628773]
 [ 0.51760269  0.51760269  0.51760269  0.51760269  0.51760269]
 [ 0.56353319  0.56353319  0.56353319  0.56353319  0.56353319]
 [ 0.55797785  0.55797785  0.55797785  0.55797785  0.55797785]
 [ 0.56893667  0.56893667  0.56893667  0.56893667  0.56893667]
 [ 0.62523183  0.62519323  0.62519323  0.62519323  0.62519323]]
[[ 0.51439037  0.51439037  0.51439037  0.51439037  0.51439037]
 [ 0.51540837  0.51540837  0.51540837  0.51540837  0.51540837]
 [ 0.5649965   0.5649965   0.5649965   0.5649965   0.5649965 ]
 [ 0.55969258  0.55969258  0.55969258  0.55969258  0.

# 3. Train models with removing 1 feature set

In [47]:
feat_sets = np.zeros(len(all_dicts),dtype=list)
for i in range(len(all_dicts)):
    feat_sets[i] = list(all_dicts[i].keys())

In [57]:
# Run model for all individual feature sets
all_C = [.01, .1, 1, 10, 100]
models = np.zeros((len(feat_sets),len(all_C)),dtype=object)
train_aucs = np.zeros((len(feat_sets),len(all_C)))
val_aucs = np.zeros((len(feat_sets),len(all_C)))
test_aucs = np.zeros((len(feat_sets),len(all_C)))
for i, ks in enumerate(feat_sets):
    for j, c in enumerate(all_C):
        print(i,j)
        # Choose feats
        fis = np.delete(np.arange(len(feat_sets)),i)
        ks2 = np.hstack(feat_sets[fis])

        # Train model
        models[i,j] = LogisticRegression(C=c)
        models[i,j].fit(X_train[ks2], y_train)
        
        # Evaluate model
        train_aucs[i,j] = roc_auc_score(y_train, models[i,j].predict_proba(X_train[ks2])[:,1])
        val_aucs[i,j] = roc_auc_score(y_val, models[i,j].predict_proba(X_val[ks2])[:,1])
        test_aucs[i,j] = roc_auc_score(y_test, models[i,j].predict_proba(X_test[ks2])[:,1])

0 0
0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4
2 0
2 1
2 2
2 3
2 4
3 0
3 1
3 2
3 3
3 4
4 0
4 1
4 2
4 3
4 4
5 0
5 1
5 2
5 3
5 4


In [58]:
print(train_aucs)
print(val_aucs)
print(test_aucs)

[[ 0.67223289  0.67223932  0.6722391   0.672239    0.67223901]
 [ 0.67214321  0.67214888  0.67214818  0.67214829  0.67214827]
 [ 0.65990239  0.65990506  0.65990456  0.65990453  0.65990452]
 [ 0.66869584  0.66870081  0.66870122  0.66870124  0.66870121]
 [ 0.66294645  0.66295187  0.66295152  0.66295141  0.66295139]
 [ 0.61180795  0.61181409  0.61181521  0.61181533  0.61181536]]
[[ 0.66709162  0.66709476  0.66709461  0.66709459  0.66709455]
 [ 0.66669234  0.66669865  0.66669914  0.66669906  0.66669904]
 [ 0.6545229   0.65452519  0.65452427  0.6545243   0.65452426]
 [ 0.66413564  0.66413138  0.66413107  0.66413108  0.66413108]
 [ 0.65722127  0.65722369  0.65722322  0.65722317  0.65722315]
 [ 0.60948606  0.60949975  0.60950181  0.60950191  0.60950195]]
[[ 0.6724719   0.67247351  0.67247238  0.67247223  0.67247219]
 [ 0.67227834  0.67227798  0.6722763   0.67227602  0.67227599]
 [ 0.66007985  0.66007708  0.66007419  0.66007394  0.66007391]
 [ 0.66899952  0.66900235  0.66900265  0.66900268  0.